# Imports

In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from imblearn.pipeline import make_pipeline

from sklearn import set_config
set_config(transform_output='pandas')

In [10]:
df = pd.read_csv('Data/cleaned_data.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_Days         312 non-null    int64  
 1   Status         312 non-null    int64  
 2   Drug           312 non-null    int64  
 3   Age            312 non-null    int64  
 4   Sex            312 non-null    int64  
 5   Ascites        312 non-null    int64  
 6   Hepatomegaly   312 non-null    int64  
 7   Spiders        312 non-null    int64  
 8   Edema          312 non-null    int64  
 9   Bilirubin      312 non-null    float64
 10  Cholesterol    284 non-null    float64
 11  Albumin        312 non-null    float64
 12  Copper         310 non-null    float64
 13  Alk_Phos       312 non-null    float64
 14  SGOT           312 non-null    float64
 15  Tryglicerides  282 non-null    float64
 16  Platelets      308 non-null    float64
 17  Prothrombin    312 non-null    float64
 18  Stage     

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,400,0,1,59,0,1,1,1,1,2.674149,5.564520,2.60,5.049856,7.448916,4.926891,5.147494,190.0,2.501436,4.0
1,4500,1,1,57,0,0,1,1,0,0.095310,5.710427,4.14,3.988984,8.908532,4.731979,4.477337,221.0,2.360854,3.0
2,1012,0,1,71,1,0,0,0,0,0.336472,5.170484,3.48,5.347108,6.246107,4.565389,4.007333,151.0,2.484907,4.0
3,1925,0,1,55,0,0,1,1,0,0.587787,5.497168,2.54,4.158883,8.719611,4.104790,4.521789,183.0,2.332144,4.0
4,1504,1,0,38,0,0,1,1,0,1.223775,5.631212,3.53,4.962845,6.508769,4.728714,4.276666,136.0,2.388763,3.0


In [11]:
target = ['Status']
y=df[target].copy()
X=df.drop(columns=target)
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)
X_train.head()

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
119,2033,1,35,1,0,0,0,0,1.252763,5.783825,3.98,6.095825,6.641182,4.869072,5.347108,344.0,2.360854,3.0
258,1615,0,60,0,0,1,0,0,1.131402,5.624018,2.97,3.737670,7.012115,4.828314,4.836282,221.0,2.282382,3.0
244,1790,0,46,0,0,1,0,0,-0.510826,5.609472,3.65,3.871201,6.677083,3.964615,5.365976,305.0,2.261763,3.0
204,2081,1,49,0,1,0,0,0,0.916291,5.945421,3.55,4.682131,7.323831,5.475208,NaN,126.0,2.332144,3.0
137,1297,1,51,1,0,1,0,0,1.987874,6.054439,3.93,5.568345,7.793174,4.981550,5.384495,252.0,2.351375,3.0


In [13]:
impute_median = SimpleImputer(strategy = 'median')
scaler = MinMaxScaler()
preprocessor = make_pipeline(impute_median, scaler)
X_train_tf = preprocessor.fit_transform(X_train)
X_test_tf = preprocessor.transfrom(X_test)
X_train_tf.head()

AttributeError: 'Pipeline' object has no attribute 'transfrom'